In [1]:
import json
import uuid
import requests

## Test embedding, context, chat completion

##### Test chat_completion

In [2]:
# Make chat_completion call
id_ = uuid.uuid4().hex
payload = {
    "id": id_,
    "system_message": "You are a helpful assistant that translates English to French.",
    "user_message": "Translate this sentence from English to French. I love programming.",
}

url = "http://localhost:8000/chat_completion"
r = requests.post(url, json=payload)
r.raise_for_status()
r.json()

{'id': '04adce65526749329c85652fc2149f71'}

In [3]:
# retrieve the response from the server

get_url = f"http://localhost:8000/get/{id_}"

r = requests.get(get_url)
r = r.json(); r

[{'id': '04adce65526749329c85652fc2149f71',
  'system_message': 'You are a helpful assistant that translates English to French.',
  'user_message': 'Translate this sentence from English to French. I love programming.',
  'response': "J'adore la programmation.",
  'request_type': 'chat_completion',
  'request_time': '1690852444.9231312',
  'response_time': '1690852448.4844308',
  'processed': True,
  'processor': '0x15d34AAf54267DB7D7c367839AAf71A00a2C6A65',
  'error': False,
  'error_message': None,
  'error_name': None}]

In [4]:
# Make multiple calls

ids = []
system_messages = [
    "You are a helpful assistant that translates English to French.",
    "You are a helpful assistant that translates English to Italian.",
    "You are a helpful assistant that translates English to Spanish.",
    "You are a helpful assistant that translates English to German.",
    "You are a helpful assistant that translates English to Russian.",
]

user_messages = [
    "Translate this sentence from English to French. I love programming.",
    "Translate this sentence from English to Italian. I love programming.",
    "Translate this sentence from English to Spanish. I love programming.",
    "Translate this sentence from English to German. I love programming.",
    "Translate this sentence from English to Russian. I love programming.",
]
request_responses = []
for i in range(5):
    id_ = uuid.uuid4().hex
    payload = {
        "id": id_,
        "system_message": system_messages[i],
        "user_message": user_messages[i],
    }
    ids.append(id_)
    url = "http://localhost:8000/chat_completion"
    r = requests.post(url, json=payload)
    request_responses.append(r)

In [5]:
# retrieve multiple responses from the server

for id_, r in zip(ids, request_responses):
    get_url = f"http://localhost:8000/get/{id_}"
    r = requests.get(get_url)
    print(r.json())

[{'id': 'a4c1c2e5d3f547b6b363204f75c08e71', 'system_message': 'You are a helpful assistant that translates English to French.', 'user_message': 'Translate this sentence from English to French. I love programming.', 'response': "J'adore la programmation.", 'request_type': 'chat_completion', 'request_time': '1690852461.5079093', 'response_time': '1690852463.8389158', 'processed': True, 'processor': '0x15d34AAf54267DB7D7c367839AAf71A00a2C6A65', 'error': False, 'error_message': None, 'error_name': None}]
[{'id': 'abb3b19594ea4cd38e3eab0d91dbcdab', 'system_message': 'You are a helpful assistant that translates English to Italian.', 'user_message': 'Translate this sentence from English to Italian. I love programming.', 'response': 'Amo la programmazione.', 'request_type': 'chat_completion', 'request_time': '1690852461.5149474', 'response_time': '1690852467.0527642', 'processed': True, 'processor': '0x15d34AAf54267DB7D7c367839AAf71A00a2C6A65', 'error': False, 'error_message': None, 'error_nam

##### Embedding

In [6]:
# Upload a file to create an embedding.
# NOTE: For pdf files, you need to parse locally and upload the text file.

emb_id_ = uuid.uuid4().hex
payload = {
    "id": emb_id_,
    "system_message": "You are a helpful assistant that translates English to French.",
    "user_message": "Translate this sentence from English to French. I love programming.",
}

# Add a file
file = open("../../README.md", "rb")
files = {"file": file}

url = "http://localhost:8000/embedding"
r = requests.post(url, files=files, data={"data":json.dumps(payload)})
r.raise_for_status()
r.json()

{'id': '3a7be8e18ab3404391454802425267b6'}

##### chat

In [7]:
# Chatting over the embedding above. Since no `memory_id` is provided, the agent will assume this is a new conversation
id_ = uuid.uuid4().hex
payload = {
    "id": id_,
    "context_id": emb_id_, # Is the same as the id when making the embedding
    "question": "What is daios?",
}

url = "http://localhost:8000/chat"
r = requests.post(url, json=payload)
r.raise_for_status()
r = r.json()
mem_id = r["memory_id"]


In [8]:
get_url = f"http://localhost:8000/get/{id_}"

r = requests.get(get_url)
r.raise_for_status()
r = r.json(); r

[{'id': '7a77137652c945acadee33d81f2f5611',
  'context_id': '3a7be8e18ab3404391454802425267b6',
  'memory_id': '195de86fdbab4ba1a88bed0442bbcb79',
  'question': 'What is daios?',
  'modified_question': None,
  'response': 'Based on the context provided, "DAIOS" refers to the Decentralized AI Operating System. It is an extension of the Open Autonomy framework and is being developed as part of a collaboration between Algovera and Valory. The goal of DAIOS is to create a decentralized operating system for AI that integrates with blockchain and messaging technologies.',
  'chat_history': [{'role': 'user', 'content': 'What is daios?'},
   {'role': 'assistant',
    'content': 'Based on the context provided, "DAIOS" refers to the Decentralized AI Operating System. It is an extension of the Open Autonomy framework and is being developed as part of a collaboration between Algovera and Valory. The goal of DAIOS is to create a decentralized operating system for AI that integrates with blockchain 

In [9]:
# since a `memory_id` is provided, the agent will use the memory to generate a follow-up response
id_ = uuid.uuid4().hex
payload = {
    "id": id_,
    "context_id": emb_id_,
    "memory_id": mem_id,
    "question": "does it provide a platform?",
}

url = "http://localhost:8000/chat"
r = requests.post(url, json=payload)
r.raise_for_status()
r.json()

{'id': '170842975dde4a879b8742583b6940f1',
 'memory_id': '195de86fdbab4ba1a88bed0442bbcb79'}

In [11]:
get_url = f"http://localhost:8000/get/{id_}"

r = requests.get(get_url)
r = r.json()
r

[{'id': '170842975dde4a879b8742583b6940f1',
  'context_id': '3a7be8e18ab3404391454802425267b6',
  'memory_id': '195de86fdbab4ba1a88bed0442bbcb79',
  'question': 'does it provide a platform?',
  'modified_question': 'Does DAIOS provide a platform?',
  'response': 'Question: Does DAIOS provide a platform?\n\nAnswer: Based on the provided context, it is not explicitly stated whether DAIOS provides a platform.',
  'chat_history': [{'role': 'user', 'content': 'What is daios?'},
   {'role': 'assistant',
    'content': 'Based on the context provided, "DAIOS" refers to the Decentralized AI Operating System. It is an extension of the Open Autonomy framework and is being developed as part of a collaboration between Algovera and Valory. The goal of DAIOS is to create a decentralized operating system for AI that integrates with blockchain and messaging technologies.'},
   {'role': 'user', 'content': 'Does DAIOS provide a platform?'},
   {'role': 'assistant',
    'content': 'Question: Does DAIOS pr

##### Bad requests are handled within the http_server

In [14]:
# non-existent memory
id_ = uuid.uuid4().hex
payload = {
    "id": id_,
    "context_id": "bf68b34c19964a73ba03c9779ef70fa9",
    "memory_id": "87b9426ab1f84ad689e0eb29e350db83",
    "question": "What is daios?",
}

url = "http://localhost:8000/chat"
r = requests.post(url, json=payload)
r.raise_for_status()
print(r.status_code)


HTTPError: 500 Server Error: Server Error for url: http://localhost:8000/chat

In [13]:
# non-existent context
id_ = uuid.uuid4().hex
payload = {
    "id": id_,
    "context_id": "bf68b34c19964a73ba03c9779ef70fa9",
    "question": "What is daios?",
}

url = "http://localhost:8000/chat"
r = requests.post(url, json=payload)
r.raise_for_status()
print(r.status_code)


HTTPError: 400 Client Error: context_id bf68b34c19964a73ba03c9779ef70fa9 does not exist for url: http://localhost:8000/chat

### V2 (Deprecated)

In [1]:
import pika
import json
import uuid

In [ ]:
credentials = pika.PlainCredentials('guest', 'guest')
parameters = pika.ConnectionParameters('localhost', 5672, credentials=credentials)
connection = pika.BlockingConnection(parameters)
channel = connection.channel()
# channel.queue_declare(queue="consume_queue")


id = uuid.uuid4().hex

# basic publish
channel.basic_publish(
    exchange='',
    routing_key='consume_queue',
    body=json.dumps({"id":id, "user_message": "What is the capital of India?"})
)

id

In [ ]:
credentials = pika.PlainCredentials('guest', 'guest')
parameters = pika.ConnectionParameters('localhost', 5672, credentials=credentials)
connection = pika.BlockingConnection(parameters)
channel = connection.channel()
# channel.queue_declare(queue="consume_queue")

# basic publish
method_frame, header_frame, body = channel.basic_get(queue="publish_queue", auto_ack=True)

body

### Deprecated

In [2]:
rmq = InteractRabbitMQ()
rmq.start()
id_ = uuid.uuid4().hex
req = {"request": "What is the capital of Indonesia", "id": id_}

In [3]:
rmq.enqueue_request(req)

In [4]:
res = rmq.get_response_queue(id_)

In [5]:
res

{'id': '3856e399bb064584a2a650c73d1f7b2c',
 'request': 'What is the capital of Indonesia',
 'response': 'The capital of Indonesia is Jakarta.'}